# Models

## machine learning model

In [ ]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import matplotlib as plt
from sklearn.model_selection import cross_val_score
import seaborn as sns

In [ ]:
models = [
    LinearSVC(class_weight= 'balanced',penalty="l2",C=.01,tol=.000001),
    SGDClassifier( random_state=109,shuffle=True),
    MultinomialNB(),
    BernoulliNB(alpha=1.0, binarize=0.0, fit_prior=True),
    LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),
    RandomForestClassifier( random_state=0),
]

CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))

cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

In [ ]:
cv_df.groupby('model_name').accuracy.mean()

In [ ]:
sns.boxplot(x='model_name', y='accuracy', data=cv_df)
sns.stripplot(x='model_name', y='accuracy', data=cv_df, 
              size=12, jitter=True, edgecolor="gray", linewidth=2)
sns.set(rc={'figure.figsize':(30,30)})

In [ ]:
# the model with best accuracy is LinearSVC , will used with our data

In [ ]:
model = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LinearSVC(random_state=0, tol=1e-5)),
                    ])

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [ ]:
target = Data["dialect"].astype('category')
print('categories: {}'.format(target.cat.categories))

In [ ]:
# train
print(metrics.classification_report(y_train, model.predict(X_train),target_names=target.cat.categories))

In [ ]:
#test
print(metrics.classification_report(y_test, y_pred,target_names=target.cat.categories))

In [ ]:
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(20,20))
sns.heatmap(conf_mat, annot=True, fmt='d',
            xticklabels=target.cat.categories, yticklabels=target.cat.categories)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average=None)

## deep learning model

In [ ]:
embedding_dim = 64
training_portion = .7
model = Sequential([

    Embedding(vocab_size, embedding_dim),
    Bidirectional(LSTM(embedding_dim)),
    #Bidirectional(LSTM(32)),
    Dense(32, activation='relu'),
    Dense(19, activation='softmax')
])
model.summary()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(train_padded, training_label_seq, epochs=5, validation_data=(validation_padded, validation_label_seq), verbose=2)